In [10]:
import openai
from latex2sympy2 import latex2sympy
import json

#model = "gpt-3.5-turbo-0613"
model = "gpt-4-0613"

def calc_latex_math_expression(expression):
    sympy_expr = latex2sympy(expression)
    result = sympy_expr.doit()
    return str(result)

functions = [
     {
      "name": "calc_latex_math_expression",
      "description": "Calculate a Latex mathematical expression",
      "parameters": {
        "type": "object",
        "properties": {
          "expression": {
            "type": "string",
            "description": "The Latex expression to calculate"
          },
        },
        "required": ["expression"]
      }
    } 
  ]

available_functions = {
    "calc_latex_math_expression": calc_latex_math_expression,
}

messages = [
    {"role": "user", "content": "If I am 42 years old and my brother is 3 years younger than me, how old is my brother? Explain step by step."},
    ]

In [7]:
def run_conversation(messages):
  # Step 1: send the conversation and available functions to GPT
  response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=functions
  )
  response_message = response["choices"][0]["message"]
  print(response_message)

  # Step 2: check if GPT wanted to call a function
  if response_message.get("function_call"):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    function_name = response_message["function_call"]["name"]
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(
        expression=function_args.get("expression"),
    )

    # Step 4: send the info on the function call and function response to GPT
    messages.append(response_message)  # extend conversation with assistant's reply
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
    )  # get a new response from GPT where it can see the function response
    return second_response

In [8]:
run_conversation(messages)

In [3]:
calc_latex_math_expression(expression="42 - 3")

'39'